In [2]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv

# **VEST MT 2020**

## **VEST Documentation**

#### Election Results

**sources**

Election results from Montana Secretary of State (https://sosmt.gov/elections/results/)

### Shapefiles

**sources**

Precinct shapefile primarily from Dave Ritts at the Montana State Library, as prepared for submission to the U.S. Census Bureau's 2020 Redistricting Data Program.

**Processing**

The three precincts in Treasure County were merged as the county transitioned to a single countywide precinct prior to the 2020 election.

## Candidate

G20PRERTRU - Donald J. Trump (Republican Party)
G20PREDBID - Joseph R. Biden (Democratic Party)
G20PRELJOR - Jo Jorgensen (Libertarian Party)

G20USSRDAI - Steve Daines (Republican Party)
G20USSDBUL - Steve Bullock (Democratic Party)

G20HALRROS - Matt Rosendale (Republican Party)
G20HALDWIL - Kathleen Williams (Democratic Party)

G20GOVRGIA - Greg Gianforte (Republican Party)
G20GOVDCOO - Mike Cooney (Democratic Party)
G20GOVLBIS - Lyman Bishop (Libertarian Party)

G20ATGRKNU - Austin Knudsen (Republican Party)
G20ATGDGRA - Raph Graybill (Democratic Party)

G20SOSRJAC - Christi Jacobsen (Republican Party)
G20SOSDBEN - Bryce Bennett (Democratic Party)

G20AUDRDOW - Troy Downing (Republican Party)
G20AUDDMOR - Shane A. Morigeau (Democratic Party)
G20AUDLROO - Roger Roots (Libertarian Party)

G20SPIRARN - Elsie Arntzen (Republican Party)
G20SPIDROM - Melissa Romano (Democratic Party)
G20SPILLEA - Kevin Leatherbarrow (Libertarian Party)


In [44]:
vest_mt_20 = gp.read_file("./raw-from-source/VEST/mt_2020/mt_2020.shp")
print(vest_mt_20.head())

  STATEFP10 COUNTYFP10    COUNTY             NAME       SOSPRECINC  \
0        30        063  Missoula        Bonner 89        BONNER 89   
1        30        063  Missoula        Bonner 92        BONNER 92   
2        30        063  Missoula    C S Porter 98      CSPORTER 98   
3        30        063  Missoula  Chief Charlo 90  CHIEF CHARLO 90   
4        30        063  Missoula       Clinton 89       CLINTON 89   

   G20PRERTRU  G20PREDBID  G20PRELJOR  G20USSRDAI  G20USSDBUL  ...  \
0          93         112           2         105         109  ...   
1         715         797          42         653         894  ...   
2         726        1218          76         677        1342  ...   
3         550         895          46         542         964  ...   
4         313         203          19         306         229  ...   

   G20ATGDGRA  G20SOSRJAC  G20SOSDBEN  G20AUDRDOW  G20AUDDMOR  G20AUDLROO  \
0         105         109         102          97         104           8   
1   

# **Load and Clean Election Results**
## **Load Election Results**

Created 2020-GeneralPrecinct-by-Precinct_Votes.csv from 2020-GeneralPrecinct-by-Precinct_Votes.xlsx by opening in Excel, removing the first header row, and saving as a csv. No file modifications were made.


In [97]:
elections_2020 = pd.read_csv("./raw-from-source/MT_SOS/2020_General_Precinct-by-Precinct.csv")

In [98]:
elections_2020.shape

(29386, 8)

In [99]:
elections_2020.head()

,CountyName,PrecinctName,RaceName,PartyCode,Votes,NameOnBallot,HouseDistrict,SenateDistrict
0,Beaverhead,PRECINCT 01,PRESIDENT,DEM,166,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36
1,Beaverhead,PRECINCT 02,PRESIDENT,DEM,181,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36
2,Beaverhead,PRECINCT 03,PRESIDENT,DEM,227,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36
3,Beaverhead,PRECINCT 04,PRESIDENT,DEM,34,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36
4,Beaverhead,PRECINCT 05,PRESIDENT,DEM,24,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36


In [100]:
office_list = ['PRESIDENT','UNITED STATES SENATOR','UNITED STATES REPRESENTATIVE','GOVERNOR & LT. GOVERNOR','ATTORNEY GENERAL','SECRETARY OF STATE','STATE AUDITOR','STATE SUPERINTENDENT OF PUBLIC INSTRUCTION']
elections_2020 = elections_2020[elections_2020["RaceName"].isin(office_list)]

In [101]:
candidate_list = ['DONALD J. TRUMP AND MICHAEL R. PENCE','JOSEPH R. BIDEN AND KAMALA D. HARRIS','JO JORGENSEN AND JEREMY "SPIKE" COHEN','STEVE DAINES','STEVE BULLOCK','MATT ROSENDALE','KATHLEEN WILLIAMS','GREG GIANFORTE AND KRISTEN JURAS','MIKE COONEY AND CASEY SCHREINER','LYMAN BISHOP AND JOHN NESPER','AUSTIN KNUDSEN','RAPH GRAYBILL','CHRISTI JACOBSEN','BRYCE BENNETT','TROY DOWNING','SHANE A MORIGEAU','ROGER ROOTS','ELSIE ARNTZEN','MELISSA ROMANO','KEVIN LEATHERBARROW']
elections_2020=elections_2020[elections_2020["NameOnBallot"].isin([x.upper() for x in candidate_list])]

In [102]:
elections_2020["NameOnBallot"].unique()

array(['JOSEPH R. BIDEN AND KAMALA D. HARRIS',
       'JO JORGENSEN AND JEREMY "SPIKE" COHEN',
       'DONALD J. TRUMP AND MICHAEL R. PENCE', 'STEVE BULLOCK',
       'STEVE DAINES', 'KATHLEEN WILLIAMS', 'MATT ROSENDALE',
       'MIKE COONEY AND CASEY SCHREINER', 'LYMAN BISHOP AND JOHN NESPER',
       'GREG GIANFORTE AND KRISTEN JURAS', 'BRYCE BENNETT',
       'CHRISTI JACOBSEN', 'RAPH GRAYBILL', 'AUSTIN KNUDSEN',
       'SHANE A MORIGEAU', 'ROGER ROOTS', 'TROY DOWNING',
       'MELISSA ROMANO', 'KEVIN LEATHERBARROW', 'ELSIE ARNTZEN'],
      dtype=object)

In [103]:
elections_2020.shape

(13260, 8)

In [114]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Montana"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
fips_dict

{'Beaverhead': '001',
 'Big Horn': '003',
 'Blaine': '005',
 'Broadwater': '007',
 'Carbon': '009',
 'Carter': '011',
 'Cascade': '013',
 'Chouteau': '015',
 'Custer': '017',
 'Daniels': '019',
 'Dawson': '021',
 'Deer Lodge': '023',
 'Fallon': '025',
 'Fergus': '027',
 'Flathead': '029',
 'Gallatin': '031',
 'Garfield': '033',
 'Glacier': '035',
 'Golden Valley': '037',
 'Granite': '039',
 'Hill': '041',
 'Jefferson': '043',
 'Judith Basin': '045',
 'Lake': '047',
 'Lewis and Clark': '049',
 'Liberty': '051',
 'Lincoln': '053',
 'McCone': '055',
 'Madison': '057',
 'Meagher': '059',
 'Mineral': '061',
 'Missoula': '063',
 'Musselshell': '065',
 'Park': '067',
 'Petroleum': '069',
 'Phillips': '071',
 'Pondera': '073',
 'Powder River': '075',
 'Powell': '077',
 'Prairie': '079',
 'Ravalli': '081',
 'Richland': '083',
 'Roosevelt': '085',
 'Rosebud': '087',
 'Sanders': '089',
 'Sheridan': '091',
 'Silver Bow': '093',
 'Stillwater': '095',
 'Sweet Grass': '097',
 'Teton': '099',
 'Toole'

In [105]:
elections_2020["STATEFP10"]= '30'
elections_2020["COUNTYFP10"]=elections_2020["CountyName"].map(fips_dict).fillna(elections_2020["CountyName"])

print(elections_2020["COUNTYFP10"].unique())

elections_2020.loc[elections_2020['CountyName'] == 'Lewis & Clark', 'CountyName'] = 'Lewis and Clark'

elections_2020["COUNTYFP10"]=elections_2020["CountyName"].map(fips_dict).fillna(elections_2020["CountyName"])

elections_2020["COUNTYFP10"].unique()




['001' '003' '005' '007' '009' '011' '013' '015' '017' '019' '021' '023'
 '025' '027' '029' '031' '033' '035' '037' '039' '041' '043' '045' '047'
 'Lewis & Clark' '051' '053' '057' '055' '059' '061' '063' '065' '067'
 '069' '071' '073' '075' '077' '079' '081' '083' '085' '087' '089' '091'
 '093' '095' '097' '099' '101' '103' '105' '107' '109' '111']


array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
       '019', '021', '023', '025', '027', '029', '031', '033', '035',
       '037', '039', '041', '043', '045', '047', '049', '051', '053',
       '057', '055', '059', '061', '063', '065', '067', '069', '071',
       '073', '075', '077', '079', '081', '083', '085', '087', '089',
       '091', '093', '095', '097', '099', '101', '103', '105', '107',
       '109', '111'], dtype=object)

In [119]:
print(elections_2020["CountyName"].nunique())
print(len(fips_dict))

56
56


In [120]:
elections_2020.head()

,CountyName,PrecinctName,RaceName,PartyCode,Votes,NameOnBallot,HouseDistrict,SenateDistrict,STATEFP10,COUNTYFP10,pivot_col
0,Beaverhead,PRECINCT 01,PRESIDENT,DEM,166,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36,30,001,001PRECINCT 01
1,Beaverhead,PRECINCT 02,PRESIDENT,DEM,181,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36,30,001,001PRECINCT 02
2,Beaverhead,PRECINCT 03,PRESIDENT,DEM,227,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36,30,001,001PRECINCT 03
3,Beaverhead,PRECINCT 04,PRESIDENT,DEM,34,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36,30,001,001PRECINCT 04
4,Beaverhead,PRECINCT 05,PRESIDENT,DEM,24,JOSEPH R. BIDEN AND KAMALA D. HARRIS,House District 72,Senate District 36,30,001,001PRECINCT 05


In [126]:
elections_2020["PrecinctName"]=elections_2020["PrecinctName"].str.strip()

elections_2020["pivot_col"]=elections_2020["COUNTYFP10"]+elections_2020["PrecinctName"]

pivoted_2020 = pd.pivot_table(elections_2020,values=["Votes"],index=["pivot_col"],columns=["NameOnBallot"],aggfunc=sum)

pivoted_2020 = pivoted_2020.fillna(0)


pivoted_2020.reset_index(drop=False,inplace=True)
pivoted_2020.columns = pivoted_2020.columns.droplevel(0)

In [127]:
pivoted_2020.head()

NameOnBallot,,AUSTIN KNUDSEN,BRYCE BENNETT,CHRISTI JACOBSEN,DONALD J. TRUMP AND MICHAEL R. PENCE,ELSIE ARNTZEN,GREG GIANFORTE AND KRISTEN JURAS,"JO JORGENSEN AND JEREMY ""SPIKE"" COHEN",JOSEPH R. BIDEN AND KAMALA D. HARRIS,KATHLEEN WILLIAMS,...,LYMAN BISHOP AND JOHN NESPER,MATT ROSENDALE,MELISSA ROMANO,MIKE COONEY AND CASEY SCHREINER,RAPH GRAYBILL,ROGER ROOTS,SHANE A MORIGEAU,STEVE BULLOCK,STEVE DAINES,TROY DOWNING
0,001PRECINCT 01,198,161,205,192,173,190,15,166,177,...,19,196,172,167,171,25,158,187,188,186
1,001PRECINCT 02,668,169,681,671,617,627,12,181,213,...,31,646,204,201,191,45,171,222,645,634
2,001PRECINCT 03,626,211,638,625,589,621,17,227,250,...,17,615,241,225,227,35,209,246,627,599
3,001PRECINCT 04,181,35,177,181,163,168,6,34,43,...,8,176,49,43,35,13,27,46,175,175
4,001PRECINCT 05,124,21,125,120,111,117,5,24,29,...,2,120,29,32,22,12,20,30,119,114


In [128]:
pivoted_2020.rename(columns={ pivoted_2020.columns[0]: "FIP_PREC" }, inplace = True)
pivoted_2020.head()

NameOnBallot,FIP_PREC,AUSTIN KNUDSEN,BRYCE BENNETT,CHRISTI JACOBSEN,DONALD J. TRUMP AND MICHAEL R. PENCE,ELSIE ARNTZEN,GREG GIANFORTE AND KRISTEN JURAS,"JO JORGENSEN AND JEREMY ""SPIKE"" COHEN",JOSEPH R. BIDEN AND KAMALA D. HARRIS,KATHLEEN WILLIAMS,...,LYMAN BISHOP AND JOHN NESPER,MATT ROSENDALE,MELISSA ROMANO,MIKE COONEY AND CASEY SCHREINER,RAPH GRAYBILL,ROGER ROOTS,SHANE A MORIGEAU,STEVE BULLOCK,STEVE DAINES,TROY DOWNING
0,001PRECINCT 01,198,161,205,192,173,190,15,166,177,...,19,196,172,167,171,25,158,187,188,186
1,001PRECINCT 02,668,169,681,671,617,627,12,181,213,...,31,646,204,201,191,45,171,222,645,634
2,001PRECINCT 03,626,211,638,625,589,621,17,227,250,...,17,615,241,225,227,35,209,246,627,599
3,001PRECINCT 04,181,35,177,181,163,168,6,34,43,...,8,176,49,43,35,13,27,46,175,175
4,001PRECINCT 05,124,21,125,120,111,117,5,24,29,...,2,120,29,32,22,12,20,30,119,114


In [129]:
pivoted_2020.insert(loc=0, column='STATEFP10', value='30')
pivoted_2020.insert(loc=1, column='COUNTYFP10', value=pivoted_2020['FIP_PREC'].str[0:3])
inv_dict = {v: k for k, v in fips_dict.items()}
print(inv_dict)
pivoted_2020.insert(loc=2, column='COUNTY', value=elections_2020["COUNTYFP10"].map(inv_dict))
pivoted_2020.insert(loc=3, column='NAME', value=pivoted_2020['FIP_PREC'].str[3:])


{'001': 'Beaverhead', '003': 'Big Horn', '005': 'Blaine', '007': 'Broadwater', '009': 'Carbon', '011': 'Carter', '013': 'Cascade', '015': 'Chouteau', '017': 'Custer', '019': 'Daniels', '021': 'Dawson', '023': 'Deer Lodge', '025': 'Fallon', '027': 'Fergus', '029': 'Flathead', '031': 'Gallatin', '033': 'Garfield', '035': 'Glacier', '037': 'Golden Valley', '039': 'Granite', '041': 'Hill', '043': 'Jefferson', '045': 'Judith Basin', '047': 'Lake', '049': 'Lewis and Clark', '051': 'Liberty', '053': 'Lincoln', '055': 'McCone', '057': 'Madison', '059': 'Meagher', '061': 'Mineral', '063': 'Missoula', '065': 'Musselshell', '067': 'Park', '069': 'Petroleum', '071': 'Phillips', '073': 'Pondera', '075': 'Powder River', '077': 'Powell', '079': 'Prairie', '081': 'Ravalli', '083': 'Richland', '085': 'Roosevelt', '087': 'Rosebud', '089': 'Sanders', '091': 'Sheridan', '093': 'Silver Bow', '095': 'Stillwater', '097': 'Sweet Grass', '099': 'Teton', '101': 'Toole', '103': 'Treasure', '105': 'Valley', '107'

In [130]:
pivoted_2020.head()

NameOnBallot,STATEFP10,COUNTYFP10,COUNTY,NAME,FIP_PREC,AUSTIN KNUDSEN,BRYCE BENNETT,CHRISTI JACOBSEN,DONALD J. TRUMP AND MICHAEL R. PENCE,ELSIE ARNTZEN,...,LYMAN BISHOP AND JOHN NESPER,MATT ROSENDALE,MELISSA ROMANO,MIKE COONEY AND CASEY SCHREINER,RAPH GRAYBILL,ROGER ROOTS,SHANE A MORIGEAU,STEVE BULLOCK,STEVE DAINES,TROY DOWNING
0,30,001,Beaverhead,PRECINCT 01,001PRECINCT 01,198,161,205,192,173,...,19,196,172,167,171,25,158,187,188,186
1,30,001,Beaverhead,PRECINCT 02,001PRECINCT 02,668,169,681,671,617,...,31,646,204,201,191,45,171,222,645,634
2,30,001,Beaverhead,PRECINCT 03,001PRECINCT 03,626,211,638,625,589,...,17,615,241,225,227,35,209,246,627,599
3,30,001,Beaverhead,PRECINCT 04,001PRECINCT 04,181,35,177,181,163,...,8,176,49,43,35,13,27,46,175,175
4,30,001,Beaverhead,PRECINCT 05,001PRECINCT 05,124,21,125,120,111,...,2,120,29,32,22,12,20,30,119,114
